# Erythroid gene lists

Ryan Dale, dalerr@niddk.nih.gov

There are multiple ways of defining "erythroid" genes. Two such methods can be found in [Li et al 2013](http://www.ncbi.nlm.nih.gov/pubmed/23610375) and [Hughes et al 2014](http://www.ncbi.nlm.nih.gov/pubmed/24413732). The goal of this notebook is 1) to compare these gene lists and 2) to get human homologs.

Comparing the gene lists should be simple, because all we have to do is look for genes that show up in each list. However, these lists contain typos, mis-identified genes, multiple synonyms for one gene, and other issues preventing them from being compared directly. This notebook documents the work needed in order to simply compare these gene lists. The output consists of lists of erythroid genes in various accessions (Ensembl, MGI symbol, Entrez) in mouse as well as human homologs. These lists are then combined to find erythroid genes unique to one study, genes defined as erythroid in both, and the union of both lists.

This notebook describes some issues with the gene list in Li et al's Table S3. Here we explain explain why those results are still valid, and why only now the issues are showing up. The genes in Table S3 were identified with a combined literature search and curation approach which was then used to annotate microarray expression results from Affymetrix Mouse 430 2.0 arrays. Genes from the array as "erythroid" if the gene symbol was found in the Table S3, and 71 of the 530 unique symbols in Table S3 were not found in the microarray results. This is not entirely unexpected since there are [known issues](http://www.ncbi.nlm.nih.gov/pubmed/16284200) when using older array designs with recent databases, and these issues were addressed by Li et al by using the latest [BrainArray CustomCDF files](http://brainarray.mbni.med.umich.edu/Brainarray/Database/CustomCDF/genomic_curated_CDF.asp). Those 71 missing genes were attributed to either a lack of probes for the gene in the original array and/or deliberate removal of problematic probesets by the BrainArray CustomCDF group. Either way, calling fewer genes as "erythroid" was a more conservative approach for the Li et al. analysis.

However, now we'd like to 1) identify homologous erythroid genes in other species and 2) compare with Hughes et al's list of gene symbols. The gene symbols are therefore being compared to the contents of multiple recent databases, and we expect every symbol to be found -- in contrast to looking for presence/absence in microarray data and expecting some fraction to be missing. When symbols are not found, we need to investigate why, and this uncovers issues in the original gene lists.

Below we fix the gene lists to maximally match current databases and compare the lists with set operations, finally saving the results for use in downstream analysis.

## Download data
The following downloads the supplemental data from Li et al and Hughes et al into a newly-created `data` directory in the current working directory.

In [1]:
%%script bash
mkdir -p "data/li-2013"
(
    cd "data/li-2013"
    curl -L -J --insecure --silent \
    "http://www.bloodjournal.org/highwire/filestream/318857/field_highwire_adjunct_files/2/TableS3.xlsx" \
    > "TableS3.xlsx"
)

In [2]:
%%script bash
mkdir -p "data/hughes-2014"
(
    cd "data/hughes-2014"
    curl -L -J --silent --insecure \
    "http://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE47758&format=file&file=GSE47758_Captured_Regions%2B500bp%2Ebed%2Egz" \
        > "GSE47758_Captured_Regions+500bp.bed.gz"
)

# Loading and cleaning Hughes 2014 data
These data are in the form of a BED file of regions captured by their assay, where the gene symbol is in the "name" column (4th column):

In [3]:
!zcat data/hughes-2014/GSE47758_Captured_Regions+500bp.bed.gz | head

chr1	16608340	16610364	Ube2w
chr1	36363177	36365656	Arid5a
chr1	59027148	59031794	Trak2
chr1	75137189	75139542	1810031K17Rik
chr1	75138735	75140516	Fam134a
chr1	75163989	75166340	Zfand2b
chr1	75174963	75177940	Abcb6
chr1	75187652	75189458	Atg9a
chr1	75215328	75217511	Tuba4a
chr1	75232055	75234432	Dnajb2


Let's load that into a pandas DataFrame:

In [4]:
import pandas
import numpy as np
hughes_table = pandas.read_table(
    'data/hughes-2014/GSE47758_Captured_Regions+500bp.bed.gz',
    compression='gzip',
    names=['chrom', 'start', 'stop', 'orig_symbol']
)
print(len(hughes_table))
hughes_table.head()

460


,chrom,start,stop,orig_symbol
0,chr1,16608340,16610364,Ube2w
1,chr1,36363177,36365656,Arid5a
2,chr1,59027148,59031794,Trak2
3,chr1,75137189,75139542,1810031K17Rik
4,chr1,75138735,75140516,Fam134a


Upon closer inspection, some genes had multiple capture regions. These genes have an underscore and an incrementing integer:

In [5]:
has_underscore = hughes_table.orig_symbol.apply(lambda x: '_' in x)
print(sum(has_underscore))
hughes_table[has_underscore].head()

37


,chrom,start,stop,orig_symbol
13,chr1,135649423,135650884,Atp2b4_1
14,chr1,135696735,135698467,Atp2b4_2
41,chr10,88968283,88970530,Nr1h4_1
42,chr10,88994894,88997607,Nr1h4_2
56,chr11,32181947,32185207,Hba-a2_1


Make a new column where these are removed:

In [6]:
hughes_table['symbol'] = hughes_table['orig_symbol'].apply(lambda x: x.split('_')[0])
hughes_table[has_underscore].head()

,chrom,start,stop,orig_symbol,symbol
13,chr1,135649423,135650884,Atp2b4_1,Atp2b4
14,chr1,135696735,135698467,Atp2b4_2,Atp2b4
41,chr10,88968283,88970530,Nr1h4_1,Nr1h4
42,chr10,88994894,88997607,Nr1h4_2,Nr1h4
56,chr11,32181947,32185207,Hba-a2_1,Hba-a2


How many genes were there with underscores?

In [7]:
x = hughes_table.groupby('symbol').orig_symbol.agg('count').sort_values()
print(x[x>1])

symbol
Ldb1        2
Sox6        2
Ptp4a3      2
Slc25a38    2
Hba-a2      2
Arhgap23    2
Atp2b4      2
Runx3       2
Lpin2       2
Nr1h4       2
Lmo2        2
Cbfa2t3     2
Asb2        2
Epb4.9      2
Mllt3       2
Fis1        2
Runx1       2
Ppp1r15a    2
Ank1        3
Ubb         3
Name: orig_symbol, dtype: int64


Were there duplicates?

In [8]:
y = hughes_table.groupby('symbol').orig_symbol.agg('nunique').sort_values()
print(y[y>1])

symbol
Slc25a38    2
Sox6        2
Ptp4a3      2
Ldb1        2
Atp2b4      2
Fis1        2
Hba-a2      2
Asb2        2
Mllt3       2
Lpin2       2
Epb4.9      2
Runx1       2
Runx3       2
Cbfa2t3     2
Nr1h4       2
Ank1        3
Name: orig_symbol, dtype: int64


Hmm, looks like `Ubb` disappeared from the list of duplicates. Maybe some others. Let's check those.

In [9]:
xs = x[x>1].index
ys = y[y>1].index
dups = ~(xs.isin(ys))
hughes_table.ix[hughes_table.symbol.isin(xs[dups])].sort_values('symbol')

,chrom,start,stop,orig_symbol,symbol
85,chr11,97300016,97303384,Arhgap23_1,Arhgap23
86,chr11,97309412,97314133,Arhgap23_1,Arhgap23
238,chr2,103795880,103799335,Lmo2_1,Lmo2
239,chr2,103808783,103811465,Lmo2_1,Lmo2
363,chr7,52780094,52782292,Ppp1r15a,Ppp1r15a
364,chr7,52780094,52782567,Ppp1r15a,Ppp1r15a
73,chr11,62363081,62366156,Ubb,Ubb
131,chr14,46703148,46704309,Ubb,Ubb
307,chr5,125869174,125870875,Ubb,Ubb


Not sure what's going on with Ubb, but the others could be missing their underscores. Since we only care about gene IDs in this analysis, let's just assume that Ubb is an erythroid gene and not worry about where it actually is.

Now we make a new dataframe of the unique symbols and copy the "symbol" column over as the index.

In [10]:
hughes_unique = pandas.DataFrame(dict(symbol=hughes_table.symbol.drop_duplicates()))
hughes_unique.index = hughes_unique.symbol
print(len(hughes_unique))
hughes_unique.head()

438


,symbol
symbol,
Ube2w,Ube2w
Arid5a,Arid5a
Trak2,Trak2
1810031K17Rik,1810031K17Rik
Fam134a,Fam134a


So while there were 460 gene IDs in the original file, after removing duplicates and collapsing those with underscores to a single gene, we have 438.

# Downloading and cleaning Li 2013 data

Read in Table S3 from Li et al (see `get-data.bash`). This is a one-column list of erythroid genes.

In [11]:
li_table = pandas.read_excel('data/li-2013/TableS3.xlsx', names=['symbol'])
print(len(li_table))
li_table.head()

533


,symbol
0,1110006O24Rik
1,1110017F19Rik
2,1110018G07Rik
3,1110018G07Rik
4,1190007F08Rik


There were some duplicates in this table; report them and de-dupe.

In [12]:
li_c = li_table.symbol.value_counts()
dups = li_c > 1
print(sum(dups), 'duplicates found:', list(li_c[dups].index))

5 duplicates found: ['1110018G07Rik', 'Stxbp4', 'Epdr2', 'Inhbc', '6430706D22Rik']


Let's remove the duplicates and make a unique dataframe like we did for the Hughes data:

In [13]:
li_unique = pandas.DataFrame(dict(symbol=li_table.symbol.drop_duplicates()))
li_unique.index = li_unique.symbol
li_unique.head()

,symbol
symbol,
1110006O24Rik,1110006O24Rik
1110017F19Rik,1110017F19Rik
1110018G07Rik,1110018G07Rik
1190007F08Rik,1190007F08Rik
1600023N17Rik,1600023N17Rik


At this point, how much overlap is there between the two lists of genes?

In [14]:
li_set = set(li_unique.symbol)
hughes_set = set(hughes_unique.symbol)
print("Li:", len(li_set))
print("Hughes:", len(hughes_set))
print("Shared:", len(li_set.intersection(hughes_set)))
print("Unique to Li et al:", len(li_set.difference(hughes_set)))
print("Unique to Hughes et al:", len(hughes_set.difference(li_set)))

Li: 528
Hughes: 438
Shared: 110
Unique to Li et al: 418
Unique to Hughes et al: 328


Hmm, not much overlap. Only a quarter to a third of genes are shared. Gene symbols can be difficult to match up correctly; let's see if we can improve this by mapping gene symbols to another kind of identifier, like Ensembl or Entrez accessions. To do this we'll use the [MyGene.info]( http://mygene.info/) service.

MyGene.info aggregates lots of databases and offers an easy-to-use but very powerful API for getting lots of information about a gene. We will be using it quite heavily.

# Attach MyGene.info data

In [15]:
import mygene
mg = mygene.MyGeneInfo()

Let's work with the Li et al data first to see what has to be done. Those identifiers look like gene symbols to me, so let's query MyGene.info and tell to look for those gene symbols:

In [16]:
results = mg.querymany(
    # the genes we're looking for
    list(li_unique.symbol),
    
    #limit to mouse
    species='mouse',
    
    # what kind of identifiers for the genes we're looking for
    scopes='symbol',
    
    # what we want to get back
    fields='symbol,ensembl.gene,entrezgene,name',
    
    # return as a pandas.DataFrame for convenience
    as_dataframe=True)

Finished.
4 input query terms found dup hits:
	[('1600023N17Rik', 2), ('1700086O06Rik', 2), ('1110006O24Rik', 2), ('1810053B23Rik', 2)]
81 input query terms found no hit:
	['1110017F19Rik', '1110018G07Rik', '1190007F08Rik', '1700051C09Rik', '2010011I20Rik', '2310047B19Rik
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


There were some genes that had multiple matches, so we'll have to figure out which one to keep.  Let's check out those duplicate hits.

In [17]:
results.ix['1810053B23Rik']

,_id,ensembl,ensembl.gene,entrezgene,name,notfound,symbol
query,,,,,,,
1810053B23Rik,ENSMUSG00000100277,NaN,ENSMUSG00000100277,NaN,RIKEN cDNA 1810053B23 gene,NaN,1810053B23Rik
1810053B23Rik,69857,NaN,NaN,69857,RIKEN cDNA 1810053B23 gene,NaN,1810053B23Rik


Strange. These duplicate hits appear to be the same gene -- the symbol is the same and the name is the same. But for some reason they're not matching up between Ensembl and Entrez. Perhaps this is an issue with the MyGene.info service (since they have different `_id` values) or maybe Entrez and Ensembl aren't talking to each other about this gene. Let's see if the other duplicate had the same issue.

In [18]:
results.ix['1700086O06Rik']

,_id,ensembl,ensembl.gene,entrezgene,name,notfound,symbol
query,,,,,,,
1700086O06Rik,ENSMUSG00000097080,NaN,ENSMUSG00000097080,NaN,RIKEN cDNA 1700086O06 gene,NaN,1700086O06Rik
1700086O06Rik,73516,NaN,NaN,73516,RIKEN cDNA 1700086O06 gene,NaN,1700086O06Rik


Yep. We'll probably have to write a merging function to handle these cases.

But the 68 genes with no hit is a little more problematic. To help resolve this, we can query MyGene.info for a specific gene to get more information about it -- maybe those missing genes are not actually "symbols" but some other accession we don't know about. Let's try searching for that first missing gene:

In [19]:
mg.query('1110017F19Rik', fields='all')

{'hits': [{'MGI': 'MGI:1915778',
   'Vega': 'OTTMUSG00000003652',
   '_id': '68528',
   'accession': {'genomic': ['AL645647', 'CH466558', 'NC_000077'],
    'protein': ['BAE33867', 'EDL34534', 'NP_001156470'],
    'rna': ['AK003745', 'AK156823', 'BC038330', 'NM_001162998']},
   'alias': '1110017F19Rik',
   'ensembl': {'gene': 'ENSMUSG00000075420',
    'protein': 'ENSMUSP00000133699',
    'transcript': 'ENSMUST00000132961'},
   'entrezgene': 68528,
   'exons': {'NM_001162998': {'cdsend': 115913559,
     'cdsstart': 115913388,
     'chr': '11',
     'exons': [[115912016, 115912456], [115913382, 115913917]],
     'strand': 1,
     'txend': 115913917,
     'txstart': 115912016}},
   'exons_mm9': {'NM_001162998': {'cdsend': 115774873,
     'cdsstart': 115774702,
     'chr': '11',
     'exons': [[115773330, 115773770], [115774696, 115775231]],
     'strand': 1,
     'txend': 115775231,
     'txstart': 115773330}},
   'genomic_pos': {'chr': '11',
    'end': 115913920,
    'start': 115912017,
 

Aha! MyGene was able to find this gene, and staring at these results, "1110017F19Rik" can be found in the "alias" field.  I wonder if we were missing a bunch of genes because we told MyGene they were symbols but they were really aliases. Let's re-run the original query, this time telling MyGene.info they could be either:

In [20]:
results = mg.querymany(
    list(li_unique.symbol),
    species='mouse',
    scopes='symbol,alias',
    fields='symbol,ensembl.gene,entrezgene,name,alias',
    as_dataframe=True)

Finished.
36 input query terms found dup hits:
	[('1600023N17Rik', 2), ('Amd1', 3), ('Elf1', 3), ('Add1', 2), ('6430706D22Rik', 2), ('Eif4e', 2), ('
9 input query terms found no hit:
	['AI4494419(Pif1)', 'Arlh2', 'BC030863', 'Dig7', 'Digap5', 'Frngtt', 'Galn10', 'LOC433237', 'Nprl3(M
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


Nice, we went from 68 missing to 9 missing. But we got a lot more duplicates (34 instead of 2) . . . let's check out some of those duplicates.

In [21]:
results.ix['Med1']

,_id,alias,ensembl,ensembl.gene,entrezgene,name,notfound,symbol
query,,,,,,,,
Med1,19014,"[AI480703, CRSP210, DRIP205, PBP, Pparbp, TRAP...",NaN,ENSMUSG00000018160,19014,mediator complex subunit 1,NaN,Med1
Med1,17193,Med1,NaN,ENSMUSG00000030322,17193,methyl-CpG binding domain protein 4,NaN,Mbd4
Med1,66070,"[0610040D20Rik, 2900052N06Rik, Ed1, c11orf5, m...",NaN,ENSMUSG00000004096,66070,CWC15 spliceosome-associated protein,NaN,Cwc15


What's happening here? MyGene found 3 hits when we asked for `Med1`. The first hit (`_id=19014`) has a matching symbol (last column). But the second row shows that "`Med1`" is also the alias for a totally different gene (`_id=17193`, `alias` column). Not sure what's going on with that last gene; my guess is that there's a "Med1" somewhere in that alias, but it's cut off in the table display here. Let's look more carefully at the alias:

In [22]:
print(list(results.ix[results['_id'] == '66070', 'alias']))

[['0610040D20Rik', '2900052N06Rik', 'Ed1', 'c11orf5', 'mED1']]


OK, I think we can discard this one because the capitialization is different ("mED1" vs "Med1"). Let's also say that we should discard the second row above because while its alias is "Med1", it's symbol is not.

How many duplicate genes do we have to go through, and how many duplicate lines are there for each?

In [23]:
c = results.index.value_counts()
print(c[c>1])
print("There are", sum(results.index.value_counts() > 1), "genes with duplicate hits.")

Mbp              6
Ank              4
Sla              3
Med1             3
Elf1             3
Pir              3
Amd1             3
Tpx2             2
1810053B23Rik    2
Tpm1             2
Bach1            2
Lmna             2
Pdk1             2
Nrf1             2
Gdpd1            2
6430706D22Rik    2
Spic             2
Usf1             2
Add1             2
Trpc2            2
1700086O06Rik    2
Ppox             2
Hrg              2
Fn3k             2
Eif4e            2
Narf             2
Tac2             2
Acp1             2
Ncoa7            2
Cit              2
1110006O24Rik    2
Por              2
Ttk              2
Mgll             2
1600023N17Rik    2
Igfbp5           2
Name: query, dtype: int64
There are 36 genes with duplicate hits.


## Resolving duplicate search results

We need to think about a general rule for which of the duplicate rows to keep. In this case, the rule could be "keep the row where the symbol matches the query".

To do this, let's ask MyGene to return a list of the duplicates and of the genes that weren't found.

In [24]:
results = mg.querymany(
    list(li_unique.symbol),
    species='mouse',
    scopes='symbol,alias',
    fields='symbol,ensembl.gene,entrezgene,name,alias',
    as_dataframe=True,
    returnall=True,
)
print("keys in results:", list(results.keys()))



# Get a list of duplicate symbols
dup_symbols = [i[0] for i in results['dup']]

# Select rows for those symbols
dups = results['out'].ix[dup_symbols]

# Identify where query ('index') matches symbol
query_matches_symbol = dups.index == dups.symbol

# Double-check that the strategy worked: after de-duping, 
# we should still have the same genes and only one row for each
orig = set(dups.index)
fixed = set(dups.index[query_matches_symbol])
assert orig == fixed
assert len(orig) == len(fixed) == len(dup_symbols)

# Drop the duplicates from the full results, and then concatenate on fixed version of the duplicates
deduped = pandas.concat((results['out'].drop(dup_symbols), dups[query_matches_symbol]))

Finished.
36 input query terms found dup hits:
	[('1600023N17Rik', 2), ('Amd1', 3), ('Elf1', 3), ('Add1', 2), ('6430706D22Rik', 2), ('Eif4e', 2), ('
9 input query terms found no hit:
	['AI4494419(Pif1)', 'Arlh2', 'BC030863', 'Dig7', 'Digap5', 'Frngtt', 'Galn10', 'LOC433237', 'Nprl3(M
keys in results: ['dup', 'missing', 'out']


Let's do some double-checks. We should have a one-to-one mapping of `_id` to `symbol` -- at least for those genes that MyGene could find.

In [25]:
print(len(li_unique), "symbols in original list")
print(len(results['missing']), "ID not found")
print(len(deduped._id.unique()), "unique IDs in results")
print(len(deduped.symbol.unique()), "unique symbols in results")
print(len(deduped), "total rows in results")

528 symbols in original list
9 ID not found
497 unique IDs in results
493 unique symbols in results
532 total rows in results


Let's take a closer look at the deduped results, focusing on cases where we have the same symbol multple times.

In [26]:
multiple = deduped.symbol.value_counts()
multiple = set(multiple[multiple > 1].index)
deduped[deduped.symbol.isin(multiple)].sort_values('symbol')

,_id,alias,ensembl,ensembl.gene,entrezgene,name,notfound,symbol
query,,,,,,,,
1110006O24Rik,66123,NaN,NaN,NaN,66123,RIKEN cDNA 1110006O24 gene,NaN,1110006O24Rik
1110006O24Rik,ENSMUSG00000107121,NaN,NaN,ENSMUSG00000107121,NaN,RIKEN cDNA 1110006O24 gene,NaN,1110006O24Rik
1600023N17Rik,69788,AV119442,NaN,NaN,69788,RIKEN cDNA 1600023N17 gene,NaN,1600023N17Rik
1600023N17Rik,ENSMUSG00000104986,NaN,NaN,ENSMUSG00000104986,NaN,RIKEN cDNA 1600023N17 gene,NaN,1600023N17Rik
1700086O06Rik,73516,NaN,NaN,NaN,73516,RIKEN cDNA 1700086O06 gene,NaN,1700086O06Rik
1700086O06Rik,ENSMUSG00000097080,NaN,NaN,ENSMUSG00000097080,NaN,RIKEN cDNA 1700086O06 gene,NaN,1700086O06Rik
1810053B23Rik,69857,"[1810008L18Rik, 1810056K14Rik]",NaN,NaN,69857,RIKEN cDNA 1810053B23 gene,NaN,1810053B23Rik
1810053B23Rik,ENSMUSG00000100277,NaN,NaN,ENSMUSG00000100277,NaN,RIKEN cDNA 1810053B23 gene,NaN,1810053B23Rik
Dfy,13349,"[AA162249, CCBP1, CD234, Darc, Dfy, ESTM35, FY...",NaN,ENSMUSG00000037872,13349,atypical chemokine receptor 1 (Duffy blood group),NaN,Ackr1


The first four rows are the duplicates we looked at before. MyGene found two different entries for each of them. That's fine, we'll deal with them later.

Other rows, however, show the same `symbol` showing up for multiple `query` entries. For example, we asked for `Darc` and `Dfy`, but MyGene tells us these are really the same gene (`Ackr1`).

Similarly, we asked for `Ankle1` and `Ankrd41`, but MyGene tells us they are the same gene (`Ankle1`).

For each set of duplicates, let's assume it doesn't really matter which symbol we choose since we're really after the `_id` column. In that case, we can use the `pandas drop_duplicates` to only keep the first row in each set of duplicate `_id`s:

In [27]:
deduped2 = deduped.drop_duplicates(subset=['_id'])

Lets re-do our check. Recall that before we had a lot more total rows than unique symbols. Now we're expecting there to be exactly as many rows as unique symbols, plus an extra 2 for those genes with duplicate hits (`1700086O06Rik` and `1810053B23Rik`).

In [28]:
print(len(li_unique), "symbols in original list")
print(len(results['missing']), "ID not found")
print(len(deduped2._id.unique()), "unique IDs in results")
print(len(deduped2.symbol.unique()), "unique symbols in results")
print(len(deduped2), "total rows in results")

528 symbols in original list
9 ID not found
497 unique IDs in results
493 unique symbols in results
497 total rows in results


Excellent! Let's recap . . .

## Recap
We queried MyGene.info using the list of genes from the supplemental table. When we told MyGene that our gene list contained "symbol", it couldn't find 68 genes. Inspecting those results, we found that some gene identifiers in our starting list were actually "alias". When we included "alias" in our search, we were able to identify another 59 genes, with 9 remaining unidentified.

This came with a cost: we got more duplicates. But we came up with a strategy to resolve those duplicates.

Then we figured out that the original gene list had synonyms in it, so we worked out how to fix that.

Next we need to work on those 9 remaining unidentified genes. For each one, we will figure out if there is a different identifier we should be using and keep track of what the new one should be.

Some of them we'll be able to assign a new name that MyGene can use; others we may have to completely remove. We'll use the `to_fix` dictionary to keep track of the former case, and the `to_remove` list for the latter.

In [29]:
to_fix = dict()
to_remove = []

Recall that above we could query MyGene for a single gene, without telling it what kind of identifier we were giving it. That's how we figured out we needed to tell MyGene we had "alias" and "symbol" in our list. For each gene, we'll try the same strategy to get more info. Failing that, we try google to figure out what gene we really mean.

## Resolving the missing genes

### AI4494419(Pif1)

In [30]:
mg.query('AI4494419(Pif1)', species='mouse', fields='accession,alias,ipi,name,refseq,reporter,symbol,unigene,uniprot')

# Try just the first part of the identifier?
mg.query('AI4494419', species='mouse', fields='accession,alias,ipi,name,refseq,reporter,symbol,unigene,uniprot')

# Nope. Last part?
res = mg.query('Pif1', species='mouse');

# That was a hit, but to simplify the output I'm showing only the relevant part.
print("symbol:", res['hits'][0]['symbol'])

symbol: Pif1


Success! Deleting the first part of the symbol seems to have worked.

Let's make sure `Pif1` is not already in our list:

In [31]:
'Pif1' in li_unique.symbol

True

Oh wait, it's already there. So we should add `AI4494419(Pif1)` to the list of genes to remove since it's a duplicate with the existing `Pif1`.

In [32]:
to_remove.append('AI4494419(Pif1)')

### Arlh2

In [33]:
mg.query('Arlh2')

{'hits': [], 'max_score': None, 'took': 2, 'total': 0}

This is a sneaky one. I googled around for a bit. I found [a patent](http://www.google.com/patents/WO2012045157A1?cl=en) that refers once to a gene "Arlh2" , but in the rest of the document it's referring to "Arih2". So there's a typo in the patent, and I think this is a typo in the supplemental table. Do we get anything for "Arih2"?

In [34]:
mg.query('Arih2', species='mouse')

{'hits': [{'_id': '23807',
   'entrezgene': 23807,
   'name': 'ariadne RBR E3 ubiquitin protein ligase 2',
   'symbol': 'Arih2',
   'taxid': 10090}],
 'max_score': 353.691,
 'took': 2,
 'total': 1}

Success! Make sure it's not in the list already:

In [35]:
print('Arih2' in li_unique.symbol)

False


Great, we'll add it to the list of gene names to fix:

In [36]:
to_fix['Arlh2'] = 'Arih2'

### BC030863

In [37]:
res = mg.query('BC030863', species='mouse', fields='all')

# Got a hit; I'm only showing the relevant entry here
print('accession:', res['hits'][0]['accession']['rna'][12])
print('symbol:', res['hits'][0]['symbol'])

accession: BC030863
symbol: Mical3


Looks like `BC030863` is the accession of a cDNA of the `Mical3` gene. Do we already have this in the original?

In [38]:
'Mical3' in li_unique.symbol

False

OK, add it to the list of genes to fix:

In [39]:
to_fix['BC030863'] = 'Mical3'

### Dig7

In [40]:
mg.query('Dig7', species='mouse', fields='all')

{'hits': [], 'max_score': None, 'took': 2, 'total': 0}

I can't find any information about `Dig7`. However, recognizing that there was a `l` -> `i` typo from `Alrh2` above, let's try the reverse typo:

In [41]:
res = mg.query('Dlg7', species='mouse', fields='all')

# Got a hit, only show relevant info
print("symbol:", res['hits'][0]['symbol'])
print("alias:", res['hits'][0]['alias'])

symbol: Dlgap5
alias: ['C77459', 'C86398', 'Dap-5', 'Dlg7', 'Hurp', 'mKIAA0008']


Let's call this a success -- one of the PubMed text reference search results talks about `Dlg7` in the context of regenerating liver.

In [42]:
print('Dlgap5' in li_unique.symbol)

to_fix['Dig7'] = 'Dlgap5'

False


### Digap5

In [43]:
mg.query('Digap5', species='mouse', fields='all');

In [44]:
res = mg.query('Dlgap5', species='mouse', fields='all')

Got a hit, only show relevant info

In [45]:
print("symbol:", res['hits'][0]['symbol'])
print("alias:", res['hits'][0]['alias'])

symbol: Dlgap5
alias: ['C77459', 'C86398', 'Dap-5', 'Dlg7', 'Hurp', 'mKIAA0008']


However, note the gene symbol is the same as the gene we were just working on.  `Dlg7` (nee `Dig7`) and `Dlgap5` (nee `Digap5`) refer to the same gene. So let's add it to the list of genes to remove from the table:

In [46]:
to_remove.append('Digap5')

### Frngtt

In [47]:
mg.query('Frngtt', species='mouse', fields='all')

{'hits': [], 'max_score': None, 'took': 1, 'total': 0}

Googling doesn't yield anything useful. Then I tried knocking off letters from the gene name:

In [48]:
mg.query('Frngt', species='mouse', fields='all')

res = mg.query('rngtt', species='mouse', fields='all')
# Got a hit, only show relevant info
print("symbol:", res['hits'][0]['symbol'])
print("alias:", res['hits'][0]['alias'])

symbol: Rngtt
alias: ['AU020997', 'HCE', 'MCE1']


Hmm, this seems questionable. The info on `Rngtt` doesn't have anything obviously erythroid-like, and "adding an extra F" doesn't seem like a common typo.

Is it already in the list?

In [49]:
'Rngtt' in li_unique.symbol

True

Ah. It's already there. Since I can't figure out what other gene it could be, let's remove this version with a "F" in front of it.

In [50]:
to_remove.append('Frngtt')

### Galn10

In [51]:
mg.query('Galn10', species='mouse', fields='all')

{'hits': [], 'max_score': None, 'took': 2, 'total': 0}

Nothing easy, and nothing showing up in google either. Let's try changing or removing characters to see if we get any hits.

In [52]:
mg.query('Gain10', species='mouse', fields='all')



mg.query('aln10', species='mouse', fields='all')



res = mg.query('Galn', species='mouse', fields='all')
# Got a hit, only show relevant info
print("symbol:", res['hits'][0]['symbol'])
print("alias:", res['hits'][0]['alias'])

symbol: Gal
alias: Galn


Wow, looking at the full output it looks like `Galn` is involved in many things. Given all its roles in the nervous system it's certainly not erythroid-specific, but maybe it's expressed in erythroid?  If we want to assume `Galn` is the name for `Galn10` then we can keep it for now, otherwise we can remove it later.

In [53]:
print('Galn' in li_unique.symbol)



to_fix['Galn10'] = 'Galn'
# to_remove.append('Galn10')

False


### LOC433237

In [54]:
mg.query('LOC433237', species='mouse', fields='all')

{'hits': [], 'max_score': None, 'took': 2, 'total': 0}

Tricky. I was able to find a `LOC433237` in the Allen Brain Atlas, which I guess MyGene.info doesn't integrate as a data source. From ABA, it says the gene symbol is `Gm9895`.

In [55]:
res = mg.query('Gm9895', species='mouse', fields='all')
print("symbol:", res['hits'][0]['symbol'])

symbol: Gm9895


Not much known here -- looking at the full output, it's a predicted ncRNA. I think we should remove it.

In [56]:
'Gm9895' in li_unique.symbol

False

In [57]:
to_remove.append('LOC433237')

### Nprl3(Mare)	

In [58]:
res = mg.query('Nprl3(Mare)', species='mouse', fields='all')

print("symbol:", res['hits'][0]['symbol'])
print("alias:", res['hits'][0]['alias'])

symbol: Nprl3
alias: ['Aag', 'CGTHBA', 'HS-26', 'HS-40', 'Mare', 'Phg', 'Prox1', 'm(alpha)RE']


This one was pretty easy. The original identifier was a mashup of the symbol and one of the aliases.

In [59]:
to_fix['Nprl3(Mare)'] = 'Nprl3'

## Recap on problematic genes
We've gone through one-by-one to figure out why genes were not being found by MyGene.info. In some cases it was a typo, in others it was a mashup of multiple identifiers, and some were duplicates. Here's what we figured out:

In [60]:
to_fix

{'Arlh2': 'Arih2',
 'BC030863': 'Mical3',
 'Dig7': 'Dlgap5',
 'Galn10': 'Galn',
 'Nprl3(Mare)': 'Nprl3'}

In [61]:
to_remove

['AI4494419(Pif1)', 'Digap5', 'Frngtt', 'LOC433237']

Let's make a function that will fix a list of genes. And then run it:

In [62]:
def fix(gene_list, to_fix, to_remove):
    """
    Converts and/or removes genes from `gene_list`.
    """
    fixed = []
    for gene in gene_list:
        if gene in to_remove:
            continue
        if gene in to_fix:
            gene = to_fix[gene]
        fixed.append(gene)
    return fixed



li_fixed = fix(li_unique.symbol, to_fix, to_remove)        

With our newly-fixed list of genes, we can re-run the query. We're still expecting to get duplicates, but we've worked out how to fix them already. We're mostly making sure that we find all genes we're looking for:

In [63]:
results = mg.querymany(
    li_fixed,
    species='mouse',
    scopes='symbol,alias',
    fields='symbol,ensembl.gene,entrezgene,name,alias',
    as_dataframe=True,
    returnall=True
)

Finished.
36 input query terms found dup hits:
	[('1600023N17Rik', 2), ('Amd1', 3), ('Elf1', 3), ('Add1', 2), ('6430706D22Rik', 2), ('Eif4e', 2), ('


Looks good -- we don't have any missing genes! Now let's create a function that performs all the de-duplicating stuff we worked out earlier. To recap, there are two kinds of "duplicates". 

1. The first is when MyGene finds multiple hits for a single gene. For example, we gave `Med1` above and MyGene found the text `Med1` in the symbol field of one gene, the alias field of another, and the text `mED1` in the alias field of another. We fix these by only keeping those where `symbol` in the result matches the query (in the `Med1` case, the gene with `symbol=Med1`).
2. The second is when different genes in the query find the same hit in MyGene (synonyms in the query). We fix these by only keeping one of the returned rows.

In [64]:
def dedup_multiple_hits(results):
    """
    Addresses the problem of multiple hits for one gene. This can 
    happen, for example, if a query shows up in multiple genes'
    alias field.
    
    This is resolved by looking at the duplicate rows, keeping only
    those where the query matches the symbol, and replacing the
    duplicate rows in results['out'] with the uniqued rows.
    
    Assumes `results` is the output from MyGene.querymany()
    using the `returnall=True` arg such that::
    
        results.keys() = ['dup', 'out', 'missing']
        
    Returns a de-duplicated DataFrame.
    """

    # Get a list of duplicate symbols
    dup_symbols = [i[0] for i in results['dup']]

    # Select rows for those symbols
    dups = results['out'].ix[dup_symbols]

    # Identify where query ('index') matches symbol
    query_matches_symbol = dups.index == dups.symbol

    # Double-check that the strategy worked: after de-duping, 
    # we should still have the same genes and only one row for each
    orig = set(dups.index)
    fixed = set(dups.index[query_matches_symbol])
    assert orig == fixed
    assert len(orig) == len(fixed) == len(dup_symbols)

    # Keep only those duplicates whose symbol matches the query
    deduped = dups[query_matches_symbol]
    
    return pandas.concat(
        (results['out'].drop(dup_symbols),
         deduped)
    )


def remove_synonyms(df):
    # Due to synonyms in the input query list, we may
    # have duplicate IDs. Just keep the first one.
    return df.drop_duplicates(subset=['_id'])


def check(orig, results, to_check):
    """
    Prints out some info about the total and unique symbols and IDs.
    
    Returns a DataFrame where there are still remaining symbols.
    """
    print(len(orig), "symbols in original list")
    print(len(results['missing']), "ID not found")
    print(len(to_check._id.unique()), "unique IDs in results")
    print(len(to_check.symbol.unique()), "unique symbols in results")
    print(len(to_check), "total rows in results")
    remaining = pandas.DataFrame()
    for _id, group in to_check.groupby('symbol'):
        if len(group) > 1:
            remaining = pandas.concat((remaining, group))
    return remaining
   



deduped = dedup_multiple_hits(results)

Our new function, `check()`, returns the cases where multiple rows have the same symbol. Let's run it on the new `deduped` dataframe. We're still expecting synonyms to be in here, since we haven't run our new `remove_synonyms` function yet.

In [65]:
check(li_fixed, results, deduped);

524 symbols in original list
0 ID not found
501 unique IDs in results
497 unique symbols in results
528 total rows in results


Now let's see how well we do after removing synonyms:

In [66]:
rm_syn = remove_synonyms(deduped)
check(li_fixed, results, rm_syn)

524 symbols in original list
0 ID not found
501 unique IDs in results
497 unique symbols in results
501 total rows in results


,_id,alias,ensembl,ensembl.gene,entrezgene,name,symbol
query,,,,,,,
1110006O24Rik,ENSMUSG00000107121,NaN,NaN,ENSMUSG00000107121,NaN,RIKEN cDNA 1110006O24 gene,1110006O24Rik
1110006O24Rik,66123,NaN,NaN,NaN,66123,RIKEN cDNA 1110006O24 gene,1110006O24Rik
1600023N17Rik,ENSMUSG00000104986,NaN,NaN,ENSMUSG00000104986,NaN,RIKEN cDNA 1600023N17 gene,1600023N17Rik
1600023N17Rik,69788,AV119442,NaN,NaN,69788,RIKEN cDNA 1600023N17 gene,1600023N17Rik
1700086O06Rik,ENSMUSG00000097080,NaN,NaN,ENSMUSG00000097080,NaN,RIKEN cDNA 1700086O06 gene,1700086O06Rik
1700086O06Rik,73516,NaN,NaN,NaN,73516,RIKEN cDNA 1700086O06 gene,1700086O06Rik
1810053B23Rik,ENSMUSG00000100277,NaN,NaN,ENSMUSG00000100277,NaN,RIKEN cDNA 1810053B23 gene,1810053B23Rik
1810053B23Rik,69857,"[1810008L18Rik, 1810056K14Rik]",NaN,NaN,69857,RIKEN cDNA 1810053B23 gene,1810053B23Rik


Not bad. We got our original two problematic symbols, but also `Mical3`. Looking closer, this should be fine -- `Mical3`  appears to have the same problem as the other two original genes: Ensembl and Entrez don't match up. So now we can write something to merge the information in each set of duplicate rows.

In [67]:
def fill_in_missing_info(x):
    """
    Given a subsetted dataframe, fill in NaNs by borrowing
    information from neighboring rows.
    
    Expects all rows in this subset to be from the same gene and
    that exactly one row's `_id` field is not an Ensembl ID.
    """
    # First back-fill any missing values, then front-fill
    x = x.fillna(method='bfill')
    x = x.fillna(method='ffill')
    keep_ind = x._id.apply(lambda j: 'ENS' not in j)
    return x[keep_ind]

def fix_duplicate_hits(x):
    """
    Subset the dataframe `x` and fill in missing info within each subset
    """
    z = x.groupby(x.index, as_index=False).apply(fill_in_missing_info)
    z.index.droplevel()
    return z

Let's run those new functions on the synonyms-removed data. Hopefully running the `check()` function will return an empty dataframe of unresolved genes, and show a one-to-one mapping of ID to symbol:

In [68]:
final = fix_duplicate_hits(rm_syn)
check(li_unique, results, final)

528 symbols in original list
0 ID not found
497 unique IDs in results
497 unique symbols in results
497 total rows in results


""


Excellent. Let's write a do-it-all function that simply calls our other functions in succession:

In [69]:
def corrected_results(results):
    """
    Runs the data wrangling functions in order:
    
        - dedup_multiple_hits
        - remove_synonyms
        - fix_duplicate_hits
    """
    deduped = dedup_multiple_hits(results)
    rm_syn = remove_synonyms(deduped)
    return fix_duplicate_hits(rm_syn).reset_index(level=0, drop=True)
    



final = corrected_results(results)

# Hughes et al 2014 genes

We'll apply the same sorts of things we learned with the Li et al data on the Hughes et al data. First let's try the easy way: assume we have gene symbols:

In [70]:
results = mg.querymany(
    list(hughes_unique.symbol),
    species='mouse',
    scopes='symbol',
    fields='symbol,ensembl.gene,entrezgene,name',
    as_dataframe=True,
    returnall=True
)

Finished.
6 input query terms found dup hits:
	[('Snhg1', 2), ('Mir144', 2), ('Cebpd', 2), ('Neat1', 2), ('1810058I24Rik', 2), ('AI662270', 2)]
26 input query terms found no hit:
	['1810031K17Rik', 'Spna1', '1110038D17Rik', 'E130309D14Rik', 'AK020764', 'Spnb1', '1110018G07Rik', '


Nope. As we saw above, manually checking genes with no hit is complex and time-consuming. Let's see if we can cut down on the number of genes with no hit by using "alias" as an additional scope, like we did before:

In [71]:
results = mg.querymany(
    list(hughes_unique.symbol),
    species='mouse',
    scopes='symbol,alias',
    fields='symbol,ensembl.gene,entrezgene,name',
    as_dataframe=True,
    returnall=True)

Finished.
24 input query terms found dup hits:
	[('Snhg1', 2), ('Mir144', 2), ('Fbxo30', 2), ('Sp2', 2), ('Mpg', 2), ('Mpl', 3), ('Cebpd', 2), ('Cat
4 input query terms found no hit:
	['AK020764', 'AK043267', '1110020G09Rik', 'AK172315']


Much better! Next we have to see if our `resolve_duplicates` works on these duplicates as well.

In [72]:
check(hughes_unique, results, results['out'])

438 symbols in original list
4 ID not found
469 unique IDs in results
463 unique symbols in results
473 total rows in results


,_id,ensembl,ensembl.gene,entrezgene,name,notfound,symbol
query,,,,,,,
1810058I24Rik,ENSMUSG00000073155,NaN,ENSMUSG00000073155,NaN,RIKEN cDNA 1810058I24 gene,NaN,1810058I24Rik
1810058I24Rik,67705,NaN,NaN,67705,RIKEN cDNA 1810058I24 gene,NaN,1810058I24Rik
AI662270,ENSMUSG00000087107,NaN,ENSMUSG00000087107,NaN,expressed sequence AI662270,NaN,AI662270
AI662270,100043636,NaN,NaN,100043636,expressed sequence AI662270,NaN,AI662270
Cebpd,ENSMUSG00000071637,NaN,ENSMUSG00000071637,NaN,"CCAAT/enhancer binding protein (C/EBP), delta",NaN,Cebpd
Cebpd,12609,NaN,NaN,12609,"CCAAT/enhancer binding protein (C/EBP), delta",NaN,Cebpd
Fn3krp,238024,NaN,ENSMUSG00000039253,238024,fructosamine 3 kinase related protein,NaN,Fn3krp
Fn3k,238024,NaN,ENSMUSG00000039253,238024,fructosamine 3 kinase related protein,NaN,Fn3krp
Mir144,ENSMUSG00000065401,NaN,ENSMUSG00000065401,NaN,microRNA 144,NaN,Mir144


It would be great if our do-it-all function would work on this. Let's try it:

In [73]:
try:
    final_hughes = corrected_results(results)
except:
    import traceback, sys
    print(traceback.format_exc())
    

Traceback (most recent call last):
  File "<ipython-input-73-f726fcd472ff>", line 2, in <module>
    final_hughes = corrected_results(results)
  File "<ipython-input-69-22be1fa25ad1>", line 9, in corrected_results
    deduped = dedup_multiple_hits(results)
  File "<ipython-input-64-01196277322d>", line 32, in dedup_multiple_hits
    assert orig == fixed
AssertionError



*Sigh*. It looks like our assertion in the `dedup_multiple_hits` failed. Let's revisit the body of that function, re-running things up to the point where the assertion failed.

In [74]:
# Get a list of duplicate symbols
dup_symbols = [i[0] for i in results['dup']]

# Select rows for those symbols
dups = results['out'].ix[dup_symbols]

# Identify where query ('index') matches symbol
query_matches_symbol = dups.index == dups.symbol

# Double-check that the strategy worked: after de-duping, 
# we should still have the same genes and only one row for each
orig = set(dups.index)
fixed = set(dups.index[query_matches_symbol])

# This assertion failed:
#assert orig == fixed

Now we check `orig` (the set of original identifiers we provided for which MyGene found multiple hits) and `fixed` (those remaining after keeping only the hit whose symbol matched the asked-for identifier). What's different?

In [75]:
set(orig).difference(fixed)

{'Beta-s'}

In [76]:
results['out'].ix['Beta-s']

,_id,ensembl,ensembl.gene,entrezgene,name,notfound,symbol
query,,,,,,,
Beta-s,100503605,NaN,ENSMUSG00000052305,100503605,"hemoglobin, beta adult s chain",NaN,Hbb-bs
Beta-s,54195,NaN,ENSMUSG00000028005,54195,"guanylate cyclase 1, soluble, beta 3",NaN,Gucy1b3


Aha! It found multiple hits for `Beta-s`, but neither of them actually had `Beta-s` in the `symbol` field.  We should be able to fix this the same way we made fixes to the Li et al data: by using a `to_fix` dictionary to re-name genes before we send them to MyGene.

So let's return to the missing genes:

In [77]:
results['missing']

['AK020764', 'AK043267', '1110020G09Rik', 'AK172315']

Let's set up our `to_fix` dictionary and our `to_remove` list, and immediately add `Beta-s`:

In [78]:
to_fix = dict()
to_remove = []
to_fix['Beta-s'] = 'Hbb-bs'

Same drill as before for figuring out how to find missing genes: we try to resolve these genes by doing a more sensitive MyGene.info query.

## AK020764

In [79]:
res = mg.query('AK020764', species='mouse', fields='all')

# Got a hit; print relevant info
print("RNA accession:", res['hits'][0]['accession']['rna'])
print("symbol:", res['hits'][0]['symbol'])

RNA accession: ['AA254104', 'AI021175', 'AK020764', 'AW909330', 'BE136127']
symbol: Mir142hg


Looking at the full output, this is a microRNA. Is this already in the list?

In [80]:
print('Mir142hg' in hughes_unique)

False


Nope, let's add it to the list of genes to fix:

In [81]:
to_fix['AK020764'] = 'Mir142hg'

## AK043267

In [82]:
res = mg.query('AK043267', species='mouse', fields='all')

# Got a hit
print("RNA accession:", res['hits'][0]['accession']['rna'])
print("symbol:", res['hits'][0]['symbol'])

RNA accession: ['AK009139', 'AK036940', 'AK043267', 'AK044589', 'AK044643', 'AK047994', 'AK077371', 'AK077603', 'AK132573', 'AK156916', 'AK172512', 'AK172537', 'BC026841', 'BC037731', 'BC057650', 'CJ155464', 'L13171', 'NM_001170537', 'NM_025282', 'XM_006517120', 'XM_006517121', 'XM_006517122', 'XM_006517123', 'XM_006517124', 'XM_006517126', 'XM_006517127', 'XM_006517128', 'XM_006517129', 'XM_006517130', 'XM_006517131', 'XM_006517132', 'XM_011244492', 'XM_011244495', 'XM_017315405', 'XM_017315406', 'XM_017315407', 'XM_017315408', 'XM_017315409', 'XM_017315410', 'XM_017315411']
symbol: Mef2c


In [83]:
print('Mef2c' in hughes_unique)

False


Another strightforward one; add it to the list:

In [84]:
to_fix['AK043267'] = 'Mef2c'

## AK172315

In [85]:
res = mg.query('AK172315', species='mouse', fields='all')

# Got a hit
print("RNA accession:", res['hits'][0]['accession']['rna'])
print("symbol:", res['hits'][0]['symbol'])

RNA accession: ['AK036383', 'AK044300', 'AK078660', 'AK083998', 'AK085774', 'AK086145', 'AK156747', 'AK172062', 'AK172315', 'BC011430', 'NR_030671']
symbol: AW011738


In [86]:
print('AW011738' in hughes_unique)

False


So far so good. Add this one too:

In [87]:
to_fix['AK172315'] = 'AW011738'

## 1110020G09Rik

In [88]:
mg.query('1110020G09Rik', species='mouse', fields='all')

{'hits': [], 'max_score': None, 'took': 1, 'total': 0}

No easy fix. Googling for this finds it in MGI -- strange that MyGene doesn't find it. This gene is also called Nadk2; NBCI calls it `BC115776` and also refers to the RIKEN cDNA name. UCSC shows the `BC115776` clone mapping to the Nadk2 gene. Based on all of that, we can infer that this identifier refers to the `Nadk2` gene. Let's make sure we can find it in MyGene:

In [89]:
res = mg.query('Nadk2', species='mouse', fields='all')
# Got a hit
print("symbol:", res['hits'][0]['symbol'])

symbol: Nadk2


In [90]:
print('Nadk2' in hughes_unique.symbol)

False


In [91]:
to_fix['1110020G09Rik'] = 'Nadk2'

OK, now we can use our `fix` function from above, which will use our `to_fix` dictionary and `to_remove` list (which happens to be empty here) to fix the query genes:

In [92]:
hughes_fixed = fix(hughes_unique.symbol, to_fix, to_remove)        

Then redo the query using the fixed names:

In [93]:
results = mg.querymany(
    list(hughes_fixed),
    species='mouse',
    scopes='symbol,alias',
    fields='symbol,ensembl.gene,entrezgene,name,alias',
    as_dataframe=True,
    returnall=True
)

Finished.
25 input query terms found dup hits:
	[('Snhg1', 2), ('Mir144', 2), ('Fbxo30', 2), ('Sp2', 2), ('Mpg', 2), ('Mpl', 3), ('AW011738', 2), ('


Good, no missing results. We still have the duplicates as expected, but we know how to remove them by now.

Let's check progress at each stage:

In [94]:
check(hughes_fixed, results, results['out']);

438 symbols in original list
0 ID not found
473 unique IDs in results
465 unique symbols in results
474 total rows in results


We have a lot of duplicates showing up; how do things change when we keep just those whose query matches symbol?

In [95]:
deduped = dedup_multiple_hits(results)
check(hughes_fixed, results, deduped);

438 symbols in original list
0 ID not found
446 unique IDs in results
438 unique symbols in results
446 total rows in results


Still looks like we could have either some synonyms or multiple hits. Let's remove synonyms and check the results:

In [96]:
rm_syn = remove_synonyms(deduped)
check(hughes_fixed, results, rm_syn)

438 symbols in original list
0 ID not found
446 unique IDs in results
438 unique symbols in results
446 total rows in results


,_id,alias,ensembl,ensembl.gene,entrezgene,name,symbol
query,,,,,,,
1810058I24Rik,ENSMUSG00000073155,NaN,NaN,ENSMUSG00000073155,NaN,RIKEN cDNA 1810058I24 gene,1810058I24Rik
1810058I24Rik,67705,"[AI414869, AI848234]",NaN,NaN,67705,RIKEN cDNA 1810058I24 gene,1810058I24Rik
AI662270,ENSMUSG00000087107,NaN,NaN,ENSMUSG00000087107,NaN,expressed sequence AI662270,AI662270
AI662270,100043636,AW496474,NaN,NaN,100043636,expressed sequence AI662270,AI662270
AW011738,ENSMUSG00000078349,NaN,NaN,ENSMUSG00000078349,NaN,expressed sequence AW011738,AW011738
AW011738,100382,NaN,NaN,NaN,100382,expressed sequence AW011738,AW011738
Cebpd,ENSMUSG00000071637,NaN,NaN,ENSMUSG00000071637,NaN,"CCAAT/enhancer binding protein (C/EBP), delta",Cebpd
Cebpd,12609,c/EBPdelta,NaN,NaN,12609,"CCAAT/enhancer binding protein (C/EBP), delta",Cebpd
Mir142hg,ENSMUSG00000084796,NaN,NaN,ENSMUSG00000084796,NaN,Mir142 host gene (non-protein coding),Mir142hg


Looks like a lot of the same problem we've been seeing before: separate entries for each gene. Luckily we have the machinery in place to fix this; let's run our do-it-all function and check the unresolved genes:

In [97]:
final_hughes = corrected_results(results)
check(hughes_fixed, results, final_hughes)

438 symbols in original list
0 ID not found
438 unique IDs in results
438 unique symbols in results
438 total rows in results


""


Victory! Let's clean things up and write a function so we can 1) provide a list of fields we want to query MyGene for, and 2) get a final dataframe for everything.

In [98]:
def dataframe_from_genes(genes, fields):
    if 'symbol' not in fields:
        raise ValueError("Deduping needs 'symbol' to be in fields")
        
    return corrected_results(
        mg.querymany(
            genes,
            species='mouse',
            scopes='symbol,alias',
            fields=fields,
            as_dataframe=True,
            returnall=True,
            verbose=False
        )
    )

With everything in place now, we can re-generate the final fixed and de-duplicated dataframes from both gene lists like this:

In [99]:
fields = 'symbol,ensembl.gene,entrezgene,name,alias'
li = dataframe_from_genes(li_fixed, fields)
hughes = dataframe_from_genes(hughes_fixed, fields)

Let's check the intersection again. For reference, here's what we started with by doing the easy thing -- intersecting the gene symbols as provided by each paper:

In [100]:
li_set = set(li_unique.symbol)
hughes_set = set(hughes_unique.symbol)
print("Li:", len(li_set))
print("Hughes:", len(hughes_set))
print("Shared:", len(li_set.intersection(hughes_set)))
print("Unique to Li et al:", len(li_set.difference(hughes_set)))
print("Unique to Hughes et al:", len(hughes_set.difference(li_set)))

Li: 528
Hughes: 438
Shared: 110
Unique to Li et al: 418
Unique to Hughes et al: 328


And here's the new version:

In [101]:
li_set = set(li.symbol)
hughes_set = set(hughes.symbol)
print("Li:", len(li_set))
print("Hughes:", len(hughes_set))
print("Shared:", len(li_set.intersection(hughes_set)))
print("Unique to Li et al:", len(li_set.difference(hughes_set)))
print("Unique to Hughes et al:", len(hughes_set.difference(li_set)))

Li: 497
Hughes: 438
Shared: 114
Unique to Li et al: 383
Unique to Hughes et al: 324


It actually didn't change that much -- we're still sitting at about 1/3 overlap between these gene lists. But the important thing is that we now have lists of genes -- complete with symbol, Ensembl, and Entrez accessions -- that we can use for downstream work.

Let's get some final lists together.

In [102]:
def get_human_genes(ids):
    """
    Given a list of MyGene.info IDs:
    
        - Query MyGene.info to retrieve Homologene entries
        - extract human homologs from the Homologene results
        - re-query MyGene.info using the human homologs, and attach
          additional fields
        - return the human homolog dataframe
        
    If `ids` is a DataFrame, then use the `_id` field. Otherwise
    expect a list of MyGene.info IDs.
    """
    
    if isinstance(ids, pandas.DataFrame):
        ids = list(ids._id)
    
    # Just ask for the homologene genes
    results = mg.querymany(
        ids,
        fields='homologene.genes',
        species='mouse',
        as_dataframe=True,
        verbose=False,
    )
    
    # Homologene results come back as lists of items. Multiple orthologs 
    # from the same species will manifest as multiple entries in the list.
    human_orthologs = []
    human_taxon_id = 9606
    for x in results['homologene.genes']:
        if isinstance(x, list):
            for i in x:
                if i[0] == human_taxon_id:
                    human_orthologs.append(i[1])

    # Re-query, using the human genes.
    return mg.querymany(
        human_orthologs,
        species='human',
        fields='symbol,name,alias,entrezgene,ensembl.gene',
        as_dataframe=True,
        verbose=False)
    



# intersection of both lists
found_in_both = li[li._id.isin(hughes._id)]

# union of both lists
found_in_either = pandas.concat((li, hughes)).drop_duplicates('_id')

# human orthologs of each
human_found_in_both = get_human_genes(found_in_both)
human_found_in_either = get_human_genes(found_in_either)
human_li = get_human_genes(li)
human_hughes = get_human_genes(hughes)



def export_gene_lists(df, label, prefix='gene-lists/',
                      fields=['_id', 'entrezgene', 'ensembl.gene', 'symbol', 'alias']):
    """
    For each of the specified fields, creates a new file named after
    the pattern
    
        <prefix><label>-<field>.txt

    containing the unique set of identifiers for that field.
    """

    for field in fields:
        filename = prefix + label + '-' + field + '.txt'
        df2 = df[field].dropna()
        with open(filename, 'w') as fout:
            for i in df2:
                if isinstance(i, list):
                    i = i = '|'.join(i)
                fout.write('%s\n' % i)



import os
if not os.path.exists('gene-lists'):
    os.makedirs('gene-lists')
    
export_gene_lists(li, 'li')
export_gene_lists(human_li, 'li_human_orthologs')
export_gene_lists(hughes, 'hughes')
export_gene_lists(human_hughes, 'hughes_human_orthologs')
export_gene_lists(found_in_either, 'union')
export_gene_lists(human_found_in_either, 'union_human_orthologs')
export_gene_lists(found_in_both, 'intersection')
export_gene_lists(human_found_in_both, 'intersection_human_orthologs')           

Now we have lists of genes we can use for downstream analysis, depending on which gene accession we need. Note that not all genes have all accession types, and that sometimes we can get an expansion of genes (like in the `intersection` case) where one mouse gene can have multiple human homologs:

In [103]:
%%bash
wc -l gene-lists/*

   413 gene-lists/hughes-alias.txt
   434 gene-lists/hughes-ensembl.gene.txt
   438 gene-lists/hughes-entrezgene.txt
   375 gene-lists/hughes_human_orthologs-alias.txt
   399 gene-lists/hughes_human_orthologs-ensembl.gene.txt
   421 gene-lists/hughes_human_orthologs-entrezgene.txt
   421 gene-lists/hughes_human_orthologs-_id.txt
   421 gene-lists/hughes_human_orthologs-symbol.txt
   438 gene-lists/hughes-_id.txt
   438 gene-lists/hughes-symbol.txt
   110 gene-lists/intersection-alias.txt
   113 gene-lists/intersection-ensembl.gene.txt
   114 gene-lists/intersection-entrezgene.txt
   110 gene-lists/intersection_human_orthologs-alias.txt
   110 gene-lists/intersection_human_orthologs-ensembl.gene.txt
   115 gene-lists/intersection_human_orthologs-entrezgene.txt
   115 gene-lists/intersection_human_orthologs-_id.txt
   115 gene-lists/intersection_human_orthologs-symbol.txt
   114 gene-lists/intersection-_id.txt
   114 gene-lists/intersection-symbol.txt
   477 gene-lists/li-alias.txt
   48